# Weekly Training Progress Report

- Compares actual vs planned weekly distance, longest route, and training time.
- Calculates percentage of plan completed.
- Identifies trends (above/below plan) for distance.

- uses data from a database which log the actual training and compares it to the excel training plan

In [ ]:
import duckdb
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from datetime import datetime
from data.paths import DB_PATH, EXCEL_PATH

conn = duckdb.connect(DB_PATH)

### Database part

Fetch data from a database and calculate summaries

In [ ]:
weekly_summary = conn.execute("""
    WITH weekly AS (
        SELECT
            strftime(activity_date, '%Y') AS year_id,
            strftime(activity_date, '%V') AS week_id,
            ROUND(SUM(EXTRACT(EPOCH FROM moving_time)) / 3600.0, 2) AS total_time_hours,
            DATE_TRUNC('second', AVG(avg_pace)) AS average_pace,
            SUM(distance_km) AS total_distance_km,
            MAX(distance_km) AS longest_hike,
            SUM(calories_kcal) AS total_calories,
            SUM(steps) AS total_steps,
            SUM(total_ascent) AS total_ascend_meters,
            SUM(total_descent) AS total_descent_meters,
            ROUND(AVG(min_temp_c),2) AS average_temperature,
            MIN(activity_date) AS week_start
        FROM tbl_activity
        WHERE EXTRACT(YEAR FROM activity_date) = EXTRACT(YEAR FROM CURRENT_DATE)
        GROUP BY year_id, week_id
        ORDER BY year_id, week_id DESC 
    )
    SELECT * FROM weekly;
             """).fetchdf()  # ← fetch as Pandas DataFrame

In [ ]:
weekly_comparison = conn.execute("""
    WITH weekly AS (
        SELECT
            strftime(activity_date, '%Y')   AS year_id,
            strftime(activity_date, '%V')   AS week_id,
            MIN(activity_date)              AS week_start,
            ROUND(SUM(EXTRACT(EPOCH FROM moving_time)) / 3600.0, 2) AS total_time_hours,
            MAX(distance_km)			 	AS longest_hike, 
            -- convert pace to seconds for math
            AVG(EXTRACT(EPOCH FROM CAST(avg_pace AS INTERVAL))) AS average_pace_sec,
            SUM(distance_km)                AS total_distance_km,
        FROM tbl_activity
        WHERE EXTRACT(YEAR FROM activity_date) = EXTRACT(YEAR FROM CURRENT_DATE)
        GROUP BY year_id, week_id
    )
    SELECT
        --year_id,
        week_id,
        ROUND(
        100.0 * ( longest_hike - LAG(longest_hike) OVER (ORDER BY week_start asc) )
                / NULLIF(LAG(longest_hike) OVER (ORDER BY week_start asc), 0),
        1
        ) AS pct_change_longest,
        ROUND(
        100.0 * ( total_distance_km - LAG(total_distance_km) OVER (ORDER BY week_start asc) )
                / NULLIF(LAG(total_distance_km) OVER (ORDER BY week_start asc), 0),
        1
        ) AS pct_change_total_distance,
        ROUND(
        100.0 * ( total_time_hours - LAG(total_time_hours) OVER (ORDER BY week_start asc) )
                / NULLIF(LAG(total_time_hours) OVER (ORDER BY week_start asc), 0),
        1
        ) AS pct_change_hiked_time,
        ROUND(
        100.0 * ( LAG(average_pace_sec) OVER (ORDER BY week_start asc) - average_pace_sec )
                / NULLIF(LAG(average_pace_sec) OVER (ORDER BY week_start asc), 0),
        1
        ) AS pct_change_pace,
        longest_hike,
        LAG(longest_hike) OVER (ORDER BY week_start ASC) AS prev_longest_hike,
        total_distance_km,
        LAG(total_distance_km) OVER (ORDER BY week_start asc) AS prev_distance_km,
        total_time_hours,
        LAG(total_time_hours) OVER (ORDER BY week_start asc) AS prev_time_hours,
        -- INTERVAL display for readability
        INTERVAL '1 second' * ROUND(average_pace_sec) AS average_pace,
        INTERVAL '1 second' * ROUND(LAG(average_pace_sec) OVER (ORDER BY week_start asc)) AS prev_average_pace
    FROM weekly
    ORDER BY week_start DESC;
""").fetchdf()

In [ ]:
weekly_best_time = conn.execute("""
    WITH weekly_fastest AS (
        SELECT
            strftime(activity_date, '%Y') AS year_id,
            strftime(activity_date, '%V') AS week_id,
            activity_date,
            distance_km,
            ROUND(1/(EXTRACT(EPOCH FROM CAST(avg_pace AS INTERVAL))/3600),2) AS avg_speed, 
            EXTRACT(EPOCH FROM CAST(avg_pace AS INTERVAL)) AS pace_sec, 
            avg_pace,
            avg_cadence,
            avg_stride_lenght,
            elapsed_time - moving_time AS inactive_time,
            total_ascent,
            total_descent
        FROM tbl_activity
        WHERE EXTRACT(YEAR FROM activity_date) = EXTRACT(YEAR FROM CURRENT_DATE)
    )
    SELECT
        week_id,
        distance_km,
        ROUND(distance_km / LAG(distance_km) OVER (ORDER BY year_id, week_id) * 100, 2) AS prc_prev_week_km,
        ROUND((avg_speed - LAG(avg_speed) OVER (ORDER BY year_id, week_id)) 
        / LAG(avg_speed) OVER (ORDER BY year_id, week_id) * 100, 2) AS speed_change_pct,
        avg_speed,
        -- Distances for prev week
        --LAG(distance_km) OVER (ORDER BY year_id, week_id) AS prev_week_km,
        -- Avg speed for prev week
        LAG(avg_speed) OVER (ORDER BY year_id, week_id) AS prev_week_speed,
        avg_pace,
        LAG(avg_pace) OVER (ORDER BY year_id, week_id ASC )AS prev_week_pace,
        avg_cadence,
        avg_stride_lenght,
        inactive_time,
        total_ascent,
        total_descent
    FROM weekly_fastest w1
    WHERE pace_sec = (
        SELECT MIN(pace_sec)
        FROM weekly_fastest w2
        WHERE w2.week_id = w1.week_id
    )
    ORDER BY year_id DESC, week_id DESC;
""").fetch_df()

### Excel part

Compare reality with a plan from excel.

But first connect excel and prepare columns into a dataframe.

In [ ]:
# Read the plan
plan_df = pd.read_excel(EXCEL_PATH, sheet_name="plán", parse_dates=["pondělí"], engine="openpyxl", usecols="A:H") 

for col in ["nejdelší trasa", "celkem týden", "doba v minutách", "doba v hodinách"]:
    plan_df[col] = plan_df[col].astype(str).str.replace(",", ".").astype(float)

plan_df['year_id_plan'] = plan_df['pondělí'].dt.year
plan_df = plan_df.dropna(subset=['year_id_plan'])

# Rename columns for convenience
plan_df = plan_df.rename(columns={
    "pondělí": "date",
    "týden roku": "week_id_plan",
    "tréninkový týden": "training_week",
    "nejdelší trasa": "longest_route_km_plan",
    "celkem týden": "total_distance_km_plan",
    "zbývá denně ujít": "remaining_daily_km",
    "doba v minutách": "time_min_plan",
    "doba v hodinách": "time_hours_plan"
})

# only to have a future plan shown (split into two so I can also compare past plans)
current_year = datetime.now().year
current_week = datetime.now().isocalendar().week

plan_future_df = plan_df[
    (plan_df['year_id_plan'] == current_year) &
    (plan_df['week_id_plan'] >= current_week)
].copy()

Ensure datatypes are correct and same for calculations - issues on year_id_plan = float + NaN (NaN dropped earlier)

In [ ]:
# Ensure year_id is int in both DataFrames
plan_df['year_id_plan'] = plan_df['year_id_plan'].astype(int)
weekly_summary['year_id'] = weekly_summary['year_id'].astype(int)

# Also ensure week_id is int
plan_df['week_id_plan'] = plan_df['week_id_plan'].astype(int)
weekly_summary['week_id'] = weekly_summary['week_id'].astype(int)

Merge dataframes together to one for comparison and make some basic calculations.

In [ ]:
comparison_df = pd.merge(
    weekly_summary,
    plan_df,
    left_on=['year_id', 'week_id'],
    right_on=['year_id_plan', 'week_id_plan'],  # your Excel column
    suffixes=('_actual', '_plan')
)

comparison_df = comparison_df[[
    "week_id","week_start",
    "total_distance_km", "total_distance_km_plan", 
    "longest_hike", "longest_route_km_plan",
    "total_time_hours", "time_hours_plan"
]]

# % of planned total distance
comparison_df['pct_distance'] = 100 * comparison_df['total_distance_km'] / comparison_df['total_distance_km_plan']

# % of planned longest hike
comparison_df['pct_longest_hike'] = 100 * comparison_df['longest_hike'] / comparison_df['longest_route_km_plan']

# % of planned time
comparison_df['pct_time'] = 100 * comparison_df['total_time_hours'] / comparison_df['time_hours_plan']

# trend based on distance
comparison_df['distance_trend'] = comparison_df['pct_distance'].apply(lambda x: 'above plan' if x >= 100 else 'below plan')

# Summaries and analyses

### Weekly summary
* shows only a summary of the week's progress.

In [ ]:
weekly_summary.head()

### Weekly summary comparison
* compares the actual past training between weeks to show trends and improvements or space for improvements.

In [ ]:
weekly_comparison.head()

### Weekly fastest activity

* shows fastest activity per week

In [ ]:
weekly_best_time.head()

### Weekly plan
* what has been planned in training plan in excel for past and following weeks.

In [ ]:
plan_future_df.head()


### Plan vs. reality
* comparison of a plan vs reality to see if I am adhering to the plan an/or by how much.

In [ ]:
# show only key columns in a readable table
display(comparison_df[['week_id','week_start', 'total_distance_km', 'total_distance_km_plan',
                       'pct_distance', 'longest_hike', 'longest_route_km_plan',
                       'pct_longest_hike', 'total_time_hours', 'time_hours_plan', 'pct_time',
                       'distance_trend']].round(1))

### Weekly distance
* a graph to show if I am adhering to a planned kilometers or not.

In [ ]:
try:
    plt.figure(figsize=(12,6), facecolor="#252526")  # figure background

    weeks = comparison_df['week_id']

    plt.plot(weeks, comparison_df['total_distance_km_plan'],
            label='Planned',
            marker='o',
            markersize=8,
            markerfacecolor='white',
            markeredgecolor='blue',
            color='blue',
            linestyle='-',  # solid line
            linewidth=2)

    plt.plot(weeks, comparison_df['total_distance_km'],
            label='Actual',
            marker='s',
            markersize=8,
            markerfacecolor='orange',
            markeredgecolor='darkorange',
            color='orange',
            linestyle='--',  # dashed line
            linewidth=2)

    plt.xlabel('Week', fontsize=12, color='white')
    plt.ylabel('Distance (km)', fontsize=12, color='white')
    plt.title('Weekly Distance: Actual vs Planned', fontsize=14, weight='bold', color='white')
    plt.xticks(color='white')
    plt.yticks(color='white')
    plt.legend(facecolor='#555555', edgecolor='white', labelcolor='white')
    plt.grid(True, linestyle='--', alpha=0.5, color='white')

    # axes background color
    plt.gca().set_facecolor('#252526')

    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Graph not possible to render due {e}")